# **CARD0003 Marking**

#### **Getting Started**
1.   Make sure your initials are correct in Marker_Intials
2.   Click Runtime > Run All

#### **FAQs**
*   Why is the Save button different colors?
  *   The big Save button at the bottom will save your scores for the current question.
  *   You can click the save button at anytime to overwrite your previous scores.
  *   Different colors:
      *   Red = you have never saved a score for this question
      *   Orange = you have previously saved a score for this question
      *   Green = you have just saved a score, and can move to a new question



*   Can I leave and mark again later?
  *   Your scores are automatically saved so long as you click the submit button.

*   Can I go back and edit my marks?
  *   Yes, you can navigate back to an old question and change your marks, make sure to click the save button.






















In [2]:
#@title Check your Initials are Correct
import pandas as pd
root_folder = 'drive/MyDrive/CARD0003/June_2024'
marking_groups = 'Marking Groups.xlsx'
marking_groups = pd.read_excel(f'{root_folder}/reference/{marking_groups}').iloc[2:]
marking_groups.columns = ['Q','mark','marker1','marker2']
Marker_Initials = 'VM' #@param ["VM", "AC"]
question_selection = list(marking_groups.loc[(marking_groups['marker1'] == Marker_Initials) | (marking_groups['marker2'] == Marker_Initials)].Q.values)
results_path = f'{root_folder}/results/scores_{Marker_Initials.upper()}.csv'

In [3]:
#@title Code
from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

import re
import pandas as pd
from IPython.display import display, Markdown, clear_output
from IPython.display import clear_output
import pandas as pd
import ipywidgets as widgets
import numpy as np
import os
import functools
import re

def format_string(string):
    if len(string.replace('\n', '')) > 90:
        # Split the string at the closest space to the 90 characters mark
        formatted_string = re.sub(r'(.{1,90})\s', r'\1\n', string)
        return formatted_string
    else:
        return string



class Counter:
    def __init__(self, value):
        self.value = value

    def increment(self, amount=1):
        self.value += amount

    def decrease(self, amount=1):
        self.value -= amount

    def set_value(self, value):
        self.value = value

class Dropdown:
    def __init__(self):
        self.values = np.zeros(len(candidates))
    def set_values(self, dropdowns):
        self.values = [dropdown.value for dropdown in dropdowns]


def next_button_clicked(b, drop, counter):
    clear_output()
    counter.increment()
    if counter.value < len(questions):
      drop.set_values(show_question(counter, drop))
      show(counter)
    else:
      questions_left = get_questions_left()
      if len(questions_left) == 0:
        print('\n\n\033[1m' + '-'* 23 + 'You have reached the end of the questions!!!' + '-'* 23  + '\n')
      else:
        print(f"Some questions are unmarked: {len(questions_left)}/{len(questions)}")
        for question in questions_left:
          print(question)

      counter.value = len(questions) - 1
      show(counter, save = False)

def prev_button_clicked(b,drop, counter):
    if counter.value > 0:
      clear_output()
      counter.decrease()
      drop.set_values(show_question(counter,drop))
      show(counter)


def save_button_clicked(b, drop, counter):
      unmarked = get_unmarked(questions[counter.value])
      if unmarked > 0:
        b.description = f'Unmarked ({unmarked}/{len(candidates)})'
      else:
        b.button_style = 'success'
        b.description = 'Saved!'
        save_results(drop)

def save_results(drop):
  results = pd.read_csv(results_path)
  results[questions[counter.value]] = drop.values
  results.to_csv(results_path, index = False)


def question_change(change,drop, counter):
    value = -1
    try:
        value = int(change.new)
    except:
        pass
    if value >= 0:
        clear_output()
        counter.set_value(value)
        drop.set_values(show_question(counter,drop))
        show(counter)

def response_change(change, drop, counter):
    drop.set_values(dropdowns)
    save_results(drop)

def get_questions_left():
    results = pd.read_csv(results_path)
    questions_left = list(set(questions).difference(results.columns))
    left = []
    question_results = results.iloc[:,1:]
    for q in question_results.columns:
      if get_unmarked(q) > 0:
        left.append(q)
    questions_left += left
    return questions_left

def show_question(counter, drop):
    global dropdowns

    dropdowns = []
    current_q = questions[counter.value]
    q =format_string(formatted_questions[counter.value])
    print(f"{Marker_Initials}. You are marking Questions: {', '.join(str(x) for x in question_selection)}")

    print('\n')
    print(f'\033[1m' + format_string(q) + ' \033[0m')
    print('\n')
    print(f'\033[1m\033[94mAnswer:\n' + answers[counter.value].replace('(2) ','(2)\n') +'\033[0m')
    print('\n\n')

    results = pd.read_csv(results_path).set_index('ID')
    results.index = results.index.astype(str)
    for candidate in candidates:
        print(f'Candidate {candidate}:\n')
        response = responses.fillna('No Response').loc[str(candidate)][current_q].replace('\n',' ').replace('\t',' ')
        response = format_string(response)
        print(response)
        value = results[current_q].loc[str(candidate)]
        dropdown =  widgets.Dropdown(options=['-'] + [str(val) for val  in np.arange(0, float(scores[current_q]) + 0.5, 0.5)],
                                     description = 'Mark:\t\t',
                                     value= str(value),
                                     style ={'description_width': '700px'},
                                     layout={'width':'800px','margin':'10px 0 25px 0px'})
        dropdown.observe(functools.partial(response_change, drop = drop, counter = counter))
        dropdowns.append(dropdown)
        display(dropdown)
        print('-' * 95)
    return dropdowns

def get_unmarked(question):
  results = pd.read_csv(results_path)
  unmarked = list(results[question].values).count('-')
  return unmarked


def show(counter, save = True):
    value = counter.value
    if value > len(candidates):
      value = len(candidates)
    unmarked = get_unmarked(questions[counter.value])
    if unmarked == 0:
        style = 'warning'
        description = "Already Marked."
        tooltip = 'Click to Save Again'
    else:
      style = 'danger'
      description = f"Unmarked {unmarked}/{len(candidates)}"
      tooltip = 'Click to Save'
    output = widgets.Output()
    save_button = widgets.Button(description=description, tooltip = tooltip, button_style=style, layout = {'font_size': '18px','width':"800px", 'height':'60px','border':'2px solid black'})
    save_button.on_click(functools.partial(save_button_clicked, drop = drop, counter = counter))

    next_button = widgets.Button(description='Next >',  tooltip = 'next question', layout = {'width':"300px", 'height':'40px','border':'1px solid black', 'margin':'20px 0 0 100px'})
    next_button.on_click(functools.partial(next_button_clicked,drop = drop, counter = counter))

    prev_button = widgets.Button(description='< Previous', tooltip = 'previous question', layout = {'width':"300px", 'height':'40px','border':'1px solid black', 'margin':'20px 100px 0 0px'})
    prev_button.on_click(functools.partial(prev_button_clicked, drop = drop, counter = counter))

    buttons = widgets.HBox([prev_button, next_button])

    if save:
      buttons = widgets.VBox([save_button, buttons, output])

    display(buttons)
    print('\n')
    forms_left = [q for q in get_questions_left()]
    forms_left = [str(form) for form in forms_left]
    print(f"\033[1mQuestions unmarked:\033[0m {len(get_questions_left())}/{len(questions)}. Qs: {', '.join(forms_left)}")
    print('\n')
    question_dropdown =  widgets.Dropdown(description='Jump to Any Question: ', value = questions[counter.value], options=questions,
                                          style={'description_width': 'initial','height':"60px"}, layout={'width': '800px'})
    question_dropdown.observe(functools.partial(question_change, drop = drop, counter = counter))
    display(question_dropdown)
    print('\n\n')

import pandas as pd

formatted_reponses = pd.read_csv(f'{root_folder}/reference/formatted_reponses.csv', index_col = 0)
formatted_questions = questions = list(formatted_reponses.columns)
formatted_questions = questions = [q for q in formatted_questions if int(q.split('Q')[-1].split('.')[0]) in question_selection]
formatted_reponses = formatted_reponses[questions]
candidates = formatted_reponses.drop(['score','answer']).astype(str).index
scores = formatted_reponses.loc['score']
answers = formatted_reponses.loc['answer']
responses = formatted_reponses.drop(['score','answer'])


## file
if os.path.isfile(results_path) :
    results = pd.read_csv(results_path)
    left = []
    for q in results.iloc[:,1:].columns:
      unmarked = get_unmarked(q)
      if  unmarked > 0:
        left.append(q)
    if len(left) == 0:
      value = 0
    else:
      value = np.where(np.array(questions) == left[0])[0][0]
else:
    results = pd.DataFrame()
    results['ID'] = candidates
    for q in questions:
      results[q] = '-'
    results.to_csv(results_path, index = False)
    value = 0



counter = Counter(value)
drop = Dropdown()
drop.set_values(show_question(counter, drop))
show(counter)


VM. You are marking Questions: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12


Form 2: Q1. i) Name the type of defect shown 


Answer:
Coronary sinus defect / Inter-atrial communication through via coronary sinus (2)



Candidate 1:

Ostium Secundum Atrial Septal Defect (ASD) is a common congenital heart defect
characterized by an opening in the atrial septum.



<ipython-input-3-f79823ec8239>:125: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'\033\033Answer:\n' + answers[counter.value].replace('(2) ','(2)\n') +'\033')


Dropdown(description='Mark:\t\t', layout=Layout(margin='10px 0 25px 0px', width='800px'), options=('-', '0.0',…

-----------------------------------------------------------------------------------------------
Candidate 2:

ASD (Atrial Septal Defect)


Dropdown(description='Mark:\t\t', layout=Layout(margin='10px 0 25px 0px', width='800px'), options=('-', '0.0',…

...

-----------------------------------------------------------------------------------------------
Candidate 15:

Secundum atrial septal defect


Dropdown(description='Mark:\t\t', layout=Layout(margin='10px 0 25px 0px', width='800px'), options=('-', '0.0',…

-----------------------------------------------------------------------------------------------




Questions unmarked: 50/50. Qs: Form 2: Q1. i) Name the type of defect shown, Form 3: Q1. ii) Describe TWO primary anatomical characteristics of this type of defect, Form 4: Q1. iii) What would be the direction of shunting (of blood) in this patient?, Form 5: Q1. iv) Which TWO factors control the direction of shunting?, Form 6: Q1. v) Which ventricle would become dilated?, Form 7: Q1. vi) What would be the direction of shunting (in diastole and systole) in a patient with the anatomy shown AND a fibrotic right ventricle with good systolic function and no tricuspid regurgita..., Form 8: Q2. i) What additional anatomic information is needed in order to distinguish the septal defect shown from a large inlet perimembranous VSD?, Form 9: Q2. ii) What would be the likely direction of shunting of blood?, Form 10: Q2. iii) What is the primary reason for the direction of shunting?, Form 11: Q3. i) Identify the structure marked 'X', Form 12: Q3. ii) Identify the structure marked ‘Y’ , Form 13: Q

Dropdown(description='Jump to Any Question: ', layout=Layout(width='800px'), options=('Form 2: Q1. i) Name the…